In [2]:
import numpy as np
import serial
import datetime as dt
import os
from os.path import join, exists
import time
import matplotlib.pyplot as plt
import matplotlib
from multiprocessing import Process, Queue, Event
import glob
import pandas as pd
import warnings
import sys

import subprocess
from tqdm.notebook import tqdm
import datetime

#if you want to display images as you record
import cv2
import matplotlib.pyplot as plt

from pyk4a import *

# imports from this module
from top_bottom_triggered.fast_animate import *
from top_bottom_triggered.commutator_utils import *
from top_bottom_triggered.video_io import *
from top_bottom_triggered.multicam_utils import *


# Experiment setup

## Get mouse order

In [3]:
# generate a random order for mice to run, based on today's date 
# (will be the same even, eg, 1 hour later, as long as date is the same)

mice_to_run = ['gmou77', 'gmou78', 'gmou81', 'gmou83']

today = dt.datetime.now().date()
date_hash = int(dt.datetime(today.year, today.month, today.day).timestamp())
np.random.seed(date_hash)
np.random.shuffle(mice_to_run)
print(mice_to_run)

['gmou77', 'gmou83', 'gmou81', 'gmou78']


## File name + dir, expt length

In [4]:
subject = 'tmp'
time_in_minutes = 0.5 # slightly longer than mkv to ensure complete overlap
base_path = R'D:\Jonah\Thermistor_recordings'
# base_path = R'E:\Jonah\CeAMouse'
file_suffix = ''  # disambiguate between two sessions on the same day
date = dt.datetime.now().strftime('%Y%m%d')


In [5]:
overwrite = False
path = os.path.join(base_path, f'{subject}\\{date}_{subject}')
# path = path.format(subject=subject, date=date)

if not os.path.exists(path):
    os.makedirs(path)
    print(f'Created {path}')
else:
    print(f'Path {path} exists!')

Path D:\Jonah\Thermistor_recordings\tmp\20230523_tmp exists!


# Commutator Setup

In [6]:
commutator_port = 'COM4'
sync_device_port = 'COM7'

with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino:
    ino.write('r'.encode('utf-8')) ## reset trigger counter

In [7]:
show_opto = False  # only set to true if there is a "stim" col in ino data
debug = False

In [8]:
# test serial port and check dac value
with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino:
    line = ino.readline().decode('utf-8').strip('\r\n')
    print(line)
    print(f'Data has {len(line.split(","))} elements')

307.3750,-25.6250,20.1875,0.05,-0.11,-0.05,0,3.30,0
Data has 9 elements


In [9]:
# hard-coded params -- don't chage
n_samples = 4000  # how many thermistor samples to show
q_downsample = 15  # leave at 15; how much to downsample rt output (doesnt affect saved data) (eg if 20, and ino at 1 khz, will be 50 hz)
fs = 500  # fs of the commutator teensy
commutator_fname = f'{date}_{subject}{file_suffix}.txt'
commutator_fullfile = os.path.join(path, commutator_fname)
if exists(commutator_fullfile) and not overwrite:
    raise ValueError(f'File {commutator_fullfile} exists! Add a suffix or change subject name.')
elif exists(commutator_fullfile):
    os.remove(commutator_fullfile)
else:
    pass

# Azure setup
Shouldn't really need to change this stuff

In [10]:
# 'bottom': '000343492012',  # old bottom
# 'bottom': '000693321712',  # new bottom

# 000364192012  # old top
# # new top
serial_numbers = {
    'bottom': '000693321712',
    'top': '000500221712'
}
master = 'top'  # don't change (should be top)
sync_delay,sync_delay_step = 0,500
record_processes = {}

file_prefix = os.path.join(path, f'{date}_{subject}' + file_suffix)
print(f'File will be saved to: {file_prefix}.XYZ')

File will be saved to: D:\Jonah\Thermistor_recordings\tmp\20230523_tmp\20230523_tmp.XYZ


In [11]:
# If you get an error here, try unfreezing the azures; or just unplug + re-plug them. 
camera_indexes = get_camera_indexes(serial_numbers)
print(camera_indexes)

Index:0	Serial:000693321712	Color:1.6.110	Depth:1.6.80
Index:1	Serial:000500221712	Color:1.6.110	Depth:1.6.80
{'bottom': 0, 'top': 1}


## Prep the experiment!
(Three priming cells, and then the DAQ cell)

In [30]:
# Get the header from the arduino, and save it to the file
first_line = 1  # don't change
second_line = 0  # don't change
sync_sent = 0
header_max_attempts = 10
second_line_max_attempts = 10

# Open queue to animator
thermistor_animator = ThermistorAnimator(n_samples, 500, show_opto=show_opto)

with open(commutator_fullfile, 'x') as file:
    with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino:
        ino.write('r'.encode('utf-8')) ## reset trigger counter
        reader = ReadLine(ino)
        
        # These checks get header and process it
        if first_line:
            # Ask the arduino to print the header
            ino.write('h'.encode('utf-8'))

            # Verify first line. First_line becomes false when good (ie, we're no longer on the first line)
            status, first_line, second_line, header, n_attempts, read_lines = first_line_check(header_max_attempts, reader, file=file)
            if not status:
                raise RuntimeError('Didnt receive header!')

            # Extract indices of values we're intersted in
            header_len = len(header.split(','))
            print(header)
            thermistor_animator.extract_indices_from_header(header)
            trigger_idx = [i for i,val in enumerate(header.split(',')) if val=='trigger'][0]
        
        if second_line:
            status, second_line = second_line_check(second_line_max_attempts, reader, header, n_good_thresh=10)
        if not status:
                raise RuntimeError('Number of csv''d datapoints doesnt match number of csv''d elements in header!') 
                
print('Header looks good!')

time,led1,led2,led3,led4,yaw,roll,pitch,acc_x,acc_y,acc_z,therm,dac,trigger
Header looks good!


In [31]:
interrupt_queues = {camera: Queue() for camera,ix in camera_indexes.items()}
trigger_started_event = Event()
for camera,ix in camera_indexes.items():
    if camera==master:
        k4a = PyK4A(Config(color_resolution=ColorResolution.OFF,  # RES_720P
                           depth_mode=DepthMode.NFOV_UNBINNED,
                           synchronized_images_only=False,
                           wired_sync_mode=WiredSyncMode.SUBORDINATE), device_id=ix)
        
        p = Process(target=capture_from_azure, 
                    args=(k4a, file_prefix+'.'+camera, int((time_in_minutes-0.1)*60)),
                    kwargs={
                        'display_time': True,
                        'display_frames':True,
                        'externally_triggered': True,
                        'trigger_started_event': trigger_started_event,
                        'interrupt_queue': interrupt_queues[camera]
                    })
        
    else:
        sync_delay += sync_delay_step
        k4a = PyK4A(Config(color_resolution=ColorResolution.OFF,
                           depth_mode=DepthMode.NFOV_UNBINNED,
                           synchronized_images_only=False,
                           wired_sync_mode=WiredSyncMode.SUBORDINATE,
                           subordinate_delay_off_master_usec=sync_delay), device_id=ix)

        p = Process(target=capture_from_azure, 
                    args=(k4a, file_prefix+'.'+camera, int((time_in_minutes-0.1)*60)+5),
                    kwargs={
                        'display_time': camera==False,
                        'externally_triggered': True,
                        'interrupt_queue': interrupt_queues[camera]})

    record_processes[camera] = p
    
record_processes

{'bottom': <Process name='Process-12' parent=488 initial>,
 'top': <Process name='Process-13' parent=488 initial>}

In [32]:
# Start Azures   
for camera in camera_indexes:
    if camera != master:
        record_processes[camera].start()
time.sleep(3)
record_processes[master].start()
time.sleep(3)  # these sleep's are critical, st the Azure's are ready for the first trigger when it arrives
print('Azures primed...')

Azures primed...


#### Run this cell to start data acquisition!

In [34]:
# timing vars
start_time = dt.datetime.now()
one_mindelta = dt.timedelta(minutes=1)
exp_timedelta = time_in_minutes*one_mindelta # key var to be compared against (now - start_time)


# Main DAQ loop
try:
    with open(commutator_fullfile, 'a') as file:
        with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino, serial.Serial(sync_device_port, baudrate=9600, timeout=0.1) as sync_device:
            
            # Create more efficient serial reader
            reader = ReadLine(ino)

            # Start thermistor animator
            thermistor_animator.start()
            
            # Run the experiment for the requested amt of time!
            while (dt.datetime.now() - start_time) < exp_timedelta:  
                
                # Read the current line of data
                line = reader.readline().decode('utf-8').strip('\r\n')
                
                # Remove the DEBUG output if present and debugging
                if debug:
                    line = line[:(line.find(',DEBUG:'))]  
                    
                # Check for the typical (but relatively infrequent) serial read issues
                if len(line) == 0:
                    print('Got empty line, continuing...')
                    continue
                elif len(line.split(',')) != header_len:
                    print('Got line with unexpected length (skipping):')
                    print(line)
                    continue
                    
                # Once, at the beginning, double check the trigger counter is starting at 0
                if not (sync_sent):
                    assert int(line.split(',')[trigger_idx]) == 0
                    
                # Once, assuming data looks good, start the sync device
                if not(sync_sent) and not(first_line or second_line):
                    print('sending start msg to sync device')
                    num = b"".join([packIntAsLong(int(time_in_minutes*60*30 + 300))])
                    sync_device.write(num)
                    sync_sent = 1
                    print(f'Sync device said: {sync_device.readline().decode("utf-8")}')
                      
                # Write data to file
                file.write(line)
                file.write('\n')
                    
                # Update the animator
                thermistor_animator.update(line)
                
                # Test the exception handling
#                 if trigger_started_event.is_set():
#                     raise RuntimeError('test')
                    
            # After data collection finishes, close animator queue
            print('closing animator queue')
            thermistor_animator.close()
            
            # Join the Azure processes (ie block until they finish)
            if trigger_started_event.is_set():
                print('joining az processes')
                exit_codes = [p.join() for p in record_processes.values() if p.is_alive()]
            
            # Sync device will finish on its own, but can just stop it here for convenience
#             print('Stopping sync device')
#             response = interrupt_sync_device(sync_device=sync_device)
        
            print('Done with main loop')
            
# Catch unexpected errors            
except:
    print('Exception')
    stop_azures(trigger_started_event, interrupt_queues, sync_device_port)
    
    # Stop the animator process
    print('Stopping animator')
    thermistor_animator.close()
    raise
    
finally:
    print('Done.')

sending start msg to sync device
Sync device said: 1200 sync pulses started!

Exception
halting azures
Stopping sync device
Stopping animator
Done.


RuntimeError: test

Traceback (most recent call last):
  File "C:\Users\dattalab\anaconda3\envs\pyk4a\lib\multiprocessing\queues.py", line 241, in _feed
    send_bytes(obj)
  File "C:\Users\dattalab\anaconda3\envs\pyk4a\lib\multiprocessing\connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "C:\Users\dattalab\anaconda3\envs\pyk4a\lib\multiprocessing\connection.py", line 290, in _send_bytes
    nwritten, err = ov.GetOverlappedResult(True)
BrokenPipeError: [WinError 109] The pipe has been ended
Traceback (most recent call last):
  File "C:\Users\dattalab\anaconda3\envs\pyk4a\lib\multiprocessing\queues.py", line 241, in _feed
    send_bytes(obj)
  File "C:\Users\dattalab\anaconda3\envs\pyk4a\lib\multiprocessing\connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "C:\Users\dattalab\anaconda3\envs\pyk4a\lib\multiprocessing\connection.py", line 280, in _send_bytes
    ov, err = _winapi.WriteFile(self._handle, buf, overlapped=True)

#### DEBUG
* "Cannot start process twice" --> re-run the cell where you create the Azure processes

#### DEBUG: Unfreeze Azures
Run this to send a short pulse of triggers via the syncing device to unfreeze the Azures, if necessary.

In [25]:
# unfreeze_azures(sync_device_port)

'5 sync pulses started!\r\n'

#### DEBUG: Stop Azures
Run this to interrupt the Azures if they're running (eg because you unfroze them)

In [ ]:
# for q in interrupt_queues.values(): q.put(tuple())

## Post-experiment summaries

In [50]:
data = pd.read_csv(glob.glob(os.path.join(path, '*.txt'))[0])

In [51]:
therm_over_thresh_count = ((data.therm > 900) & (data.dac<=0.1)).sum()
therm_under_thresh_count = ((data.therm < 200) & (data.dac >= 3.25)).sum()
print(f'Therm over: {therm_over_thresh_count}')
print(f'Therm under: {therm_under_thresh_count}')
print(f'Time elapsed since start: {(dt.datetime.now() - start_time).seconds/60:0.1f} minutes')

Therm over: 0
Therm under: 29038
Time elapsed since start: 5.0 minutes
